In [1]:
from sklearn.utils import shuffle
from sklearn import svm
import numpy as np
import pickle
import os
import pandas as pd
import scipy
from scipy.optimize import linear_sum_assignment
from scipy.spatial import distance
from sklearn.datasets import make_blobs
import math
import argparse
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from ucimlrepo import fetch_ucirepo 
from collections import Counter
from scipy.stats import entropy
from sklearn import tree
from sklearn import preprocessing
from itertools import product
from algs_lib import *

In [2]:
mi_range = [1.0]


train_x, train_y, test_x, test_y, num_classes, train_len = gen_iris(normalize=True)

subsample_rate = int(0.5*train_len)

C_range = [x / 100. for x in range(1, 101)]
num_trials = 500

for mi in mi_range:
    iris_noise = {}
    iris_acc = {}
    for C in C_range:
        print(f"C={C}, mi={mi}")
        
        est_noise = rand_mechanism_individual_noise(train_x, train_y, run_svm, subsample_rate, index = 0, tau=3,
                                         num_classes = num_classes, regularize=C, max_mi=mi)[2]
        iris_noise[C] = est_noise

        num_features = len(train_x[0])
        model = svm.LinearSVC(dual=False, random_state=None)
        model.fit(train_x[:50], train_y[:50])
        avg_orig_acc = 0
        avg_priv_acc = 0
        for i in range(num_trials):
            shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
            shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
            model, svm_vec = run_svm(shuffled_x1, shuffled_y1, num_classes=num_classes, seed=None,
                                     regularize=C)
            acc = model.score(test_x, test_y)
            avg_orig_acc += acc
            for ind in range(len(svm_vec)):
                c = np.random.normal(0, scale=iris_noise[C])
                svm_vec[ind] += c
            reshape_val = num_classes
            if num_classes == 2:
                reshape_val = 1 # special case for binary
            svm_mat = np.reshape(svm_vec, (reshape_val, num_features+1))
            intercept = svm_mat[:, -1]
            svm_mat = svm_mat[:, :-1]
            model.coef_ = svm_mat
            model.intercept_ = intercept
            priv_acc = model.score(test_x, test_y)
            avg_priv_acc += priv_acc
        avg_orig_acc /= num_trials
        avg_priv_acc /= num_trials
        iris_acc[C] = (avg_orig_acc, avg_priv_acc)
        print(f'acc={avg_orig_acc}, {avg_priv_acc}')

C=0.01, mi=1.0
[0.33333333 0.20833333 0.50847458 0.5       ] (1, 0)
(99, 4) (99, 2)
acc=0.6322040816326521, 0.6322857142857131
C=0.02, mi=1.0
[0.33333333 0.20833333 0.50847458 0.5       ] (1, 0)
(99, 4) (99, 2)
acc=0.6508163265306108, 0.6513469387755088
C=0.03, mi=1.0
[0.33333333 0.20833333 0.50847458 0.5       ] (1, 0)
(99, 4) (99, 2)


KeyboardInterrupt: 

In [5]:
name = 'data_rice/rice_acc_reg={}.pkl'
for r in [(0.01, 0.1, 0.56), (0.01, 0.15, 0.53),  (0.01, 0.15, 0.56), 
          (0.01, 0.2, 0.51),  (0.01, 0.2, 0.56),  (0.01, 0.35, 0.51)]:
    print(pickle.load(open(name.format(r), 'rb')))

{(0.01, 0.1, 0.56): (0.9088003502626969, 0.46035026269702284)}
{(0.01, 0.15, 0.53): (0.9117513134851144, 0.44958844133099857)}
{(0.01, 0.15, 0.56): (0.9066462346760076, 0.4394746059544662)}
{(0.01, 0.2, 0.51): (0.9051488616462343, 0.44472854640980747)}
{(0.01, 0.2, 0.56): (0.9084588441330997, 0.4715323992994744)}
{(0.01, 0.35, 0.51): (0.9002714535901923, 0.8998423817863397)}


In [5]:
name = 'iris_ind_data/iris_svm_ind_noise_mi=0.015625.pkl'

In [6]:
pickle.load(open(name, 'rb'))

{0.05: 0.2487263423749335,
 0.1: 0.3939403244525577,
 0.15: 0.5485518906856486,
 0.2: 0.7128162529787387,
 0.25: 0.882549990765237,
 0.3: 1.0418775077040243,
 0.35: 1.263174616844902,
 0.4: 1.434620214414537,
 0.45: 1.5514065856750276,
 0.5: 1.818719782182062,
 0.55: 1.873550840646252,
 0.6: 1.9953339519697801,
 0.65: 2.0695599593033887,
 0.7: 2.139677513781071,
 0.75: 2.2144174492136557,
 0.8: 2.268307557254384,
 0.85: 2.4734903852345393,
 0.9: 2.5490654428847024,
 0.95: 2.648599934979816,
 1.0: 2.6342161472083885}